<a href="https://colab.research.google.com/github/Sonico98/FFmpeg_for_Drive/blob/main/FFmpeg_for_GDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color='blue'> FFmpeg for Google Drive, with mkv transcoding support </font>**

#__1. Install FFmpeg__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/sudo-ken/FFmpeg-for-GDrive/master/res/ffmpeg.png' height="80" alt="Gdrive-logo"/></center>
#@markdown <center><h3>FFmpeg</h3></center><br>
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/sudo-ken/FFmpeg-for-GDrive/master/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)
 
loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('pip install ffpb')
os.system('add-apt-repository ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt-get install mediainfo ffmpeg')
clear_output()
print('Installation finished.')

#__2. Mount Google Drive__





In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive')
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

#__3. Run FFmpeg Scripts (Convert, Edit, Trim + more)__

###__Transcode MKV ➔ MP4 (and burn the first subs found)__

In [ ]:
import os, sys, re
from IPython.display import clear_output
 
#@markdown ##**Enter the file location**
VIDEO_PATH = "" #@param {type:"string"}
 
output_file_path = re.search("^[\/].+\/", VIDEO_PATH)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", VIDEO_PATH)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)
 
os.environ['inputFile'] = VIDEO_PATH
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw
 
# Install some basic fonts to the system. This happens only on first run.
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
textAn("Installing Fonts...", ty='twg')
if not os.path.exists('/usr/share/fonts/truetype/new'):
    os.system('mkdir /usr/share/fonts/truetype/new')
    os.system('git clone https://github.com/ElGordoEncodes/FFMPGFonts.git')
    os.system('mv FFMPGFonts/fonts/* /usr/share/fonts/truetype/new')
    os.system('rm -rf FFMPGFonts')
 
# Extract fonts embedded inside the mkv file.
!cp "$inputFile" '/usr/share/fonts/truetype/new'
for i in range(100):    # Increase in case there are more than 99 embeds inside the file
    dumper = f'cd /usr/share/fonts/truetype/new && ffmpeg -dump_attachment:t:{i} "" -i *.mkv'
    os.system(dumper)
os.system('cd /usr/share/fonts/truetype/new && rm -rf *.mkv')
os.system('fc-cache -f -v')
 
# Extract subtitles to burn. Assumes only one subtitle file is present.
textAn("Fonts installed. Extracting subtitles...", ty='twg')
os.system('mkdir /content/temp')
os.system('ffmpeg -y -i "$inputFile" "/content/temp/sub.ass"')
clear_output()
textAn("Subtitles extracted. Converting...", ty='twg')
 

#@markdown ##**<br/>Choose encoding parameters**
#@markdown ###&nbsp;&nbsp;- Video
VIDEO_CODEC = "libx264" #@param ["libx264", "libx265"] {allow-input: true}
PRESET = "medium" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "very slow", "placebo"]
CRF = 23 #@param {type:"slider", min:0, max:51, step:1}
PIXEL_FORMAT = "yuv420p" #@param ["yuv420p", "yuv422p", "yuv444p", "yuv420p10le", "yuv420p12le"] {allow-input: true}
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_Note: yuv420p10le and yuv420p12le are only available with libx265_
#@markdown ###<br/>&nbsp;&nbsp;- Audio
AUDIO_CODEC = "aac" #@param ["copy", "aac", "opus", "vorbis", "mp3", "flac", "wavpack"] {allow-input: true}
AUDIO_BITRATE = "256k" #@param {type:"string"}
SAMPLE_RATE = 48000 #@param {type:"integer"}

os.environ['vcodec'] = VIDEO_CODEC
os.environ['preset'] = PRESET
os.environ['crf'] = str(CRF)
os.environ['pixfmt'] = PIXEL_FORMAT
os.environ['acodec'] = AUDIO_CODEC
os.environ['abitrate'] = AUDIO_BITRATE
os.environ['sample_rate'] = str(SAMPLE_RATE)

os.system('cp "$inputFile" /content/temp')
!ffpb -i /content/temp/"$fileName".mkv -vf "ass=/content/temp/sub.ass" -c:a "$acodec" -strict -2 -b:a "$abitrate" -ar "$sample_rate" -c:v "$vcodec" -crf "$crf" -preset "$preset" -pix_fmt "$pixfmt" -movflags +faststart /content/temp/"$fileName".mp4

# Optional: delete original MKV file. You'll also have to delete it from your GDrive Trash.
# os.system('rm -rf "$inputFile"')
os.system('mv /content/temp/"$fileName.mp4" "$outputPath"')
os.system('rm -rf "/content/temp/')

print('Conversion finished.')

###__Display Media File Metadata__

In [ ]:
import os, sys, re

media_file_path = "" #@param {type:"string"}

os.environ['inputFile'] = media_file_path

!ffmpeg -i "$inputFile" -hide_banner

> *You can ignore the* "`At least one output file must be specified`" *error after running this.*




###__Convert Video File ➔ .mp4 (Lossless)__

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mp4

###__Convert Video File ➔ .mkv (Lossless)__

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mkv

###__Trim Video File (Lossless)__

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:01:00.000" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -c copy "$outputPath"/"$fileName"-TRIM."$fileExtension"

###__Crop Video__

<h3> Crop Variables Explanation:

* `out_width` = The width of your cropped video file.
* `out_height` = The height of your cropped video file.
* `starting_position_x` & `starting_position_y` = These values define the x & y coordinates of the top left corner of your original video to start cropping from.

###### *Example: For cropping the black bars from a video that looked like* [this](https://i.imgur.com/ud8nbvT.png):
* *For your starting coordinates* (`x` , `y`) *you would use* (`0` , `138`).
* *For* `out_width` *you would use* `1920`. *And for* `out_height` *you would use `804`.*





In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
out_width = "1920" #@param {type:"string"}
out_height = "804" #@param {type:"string"}
starting_position_x = "0" #@param {type:"string"}
starting_position_y = "138" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outWidth'] = out_width
os.environ['outHeight'] = out_height
os.environ['positionX'] = starting_position_x
os.environ['positionY'] = starting_position_y
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -filter:v "crop=$outWidth:$outHeight:$positionX:$positionY" "$outputPath"/"$fileName"-CROP."$fileExtension"

###__Extract Audio from Video File (Lossless)__

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
output_file_extension = 'm4a' #@param ["m4a", "mp3", "opus", "flac", "wav"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
output_file_path = re.search("^[\/].+\/", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = output_file_extension

!ffmpeg -hide_banner -i "$inputFile" -vn -c:a copy "$outputPath"/"$fileName"-audio."$fileType"

###__Re-encode a Video to a Different Resolution__

In [ ]:
import os, sys, re

video_file_path = '' #@param {type:"string"}
resolution = '1080p' #@param ["2160p", "1440p", "1080p", "720p", "480p", "360p", "240p"]
file_type = 'mp4' #@param ["mkv", "mp4"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
testsplit = video_file_path.split("/")
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
resolution_raw = re.search("[^p]{3,4}", resolution)
output_file_path = re.search("^[\/].+\/", video_file_path)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = file_type
os.environ['resolutionHeight'] = resolution_raw.group(0)

!ffmpeg -hide_banner -i "$inputFile" -vf "scale=-1:"$resolutionHeight"" -c:a copy -strict experimental "$outputPath"/"$fileName"-"$resolutionHeight"p."$fileType"

###__Extract Individual Frames from Video__

In [ ]:
#@markdown This will create a folder in the same directory titled "`Extracted Frames`"
#@markdown - [Example](https://i.imgur.com/yPDk1hO.png) of output folder
import os, sys, re

video_file_path = "" #@param {type:"string"}
start_time = "00:01:00.000" #@param {type:"string"}
end_time = "00:01:05.000" #@param {type:"string"}
frame_rate = "23.976" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['frameRate'] = frame_rate
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!mkdir "$outputPath"/"Extracted Frames"
!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -r "$frameRate"/1 "$outputPath"/"Extracted Frames"/frame%04d.png

###__Generate Thumbnails - Preview from Video (3x2)__

In [ ]:
#@markdown Example of output image: https://i.imgur.com/0ymP144.png <br>
import os, sys, re

video_file_path = "" #@param {type:"string"}
output_file_type = 'png' #@param ["png", "jpg"]

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outputExtension'] = output_file_type
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -vframes 1 -q:v 2 -vf "select=not(mod(n\,200)),scale=-1:480,tile=3x2" -an "$outputPath"/"$fileName"_thumbnails."$outputExtension"

###__Convert Audio Filetype (mp3, m4a, ogg, flac, etc.)__

In [ ]:
import os, sys, re

audio_file_path = "" #@param {type:"string"}
output_file_type = "mp3" #@param ["mp3", "ogg", "m4a", "opus", "flac", "alac", "wav"]

output_file_path = re.search("^[\/].+\/", audio_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", audio_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = audio_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileExtension'] = output_file_type
os.environ['fileName'] = filename_raw

!ffmpeg -hide_banner -i "$inputFile" "$outputPath"/"$fileName"converted."$fileExtension"

###__Sharable Links of Randomly Extracted Frames from Video__

In [ ]:
import os, re, time, pathlib
import urllib.request
from IPython.display import clear_output

Auto_UP_Gdrive = False  
AUTO_MOVE_PATH = "/content" 
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    runSh,
    findProcess,
    loadingAn,
    updateCheck,
    ngrok
)

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!mkdir -p "/content/frames"

for i in range(10):
    clear_output()
    loadingAn()
    print("Uploading Frames...")

%cd "/content/frames"
!ffmpeg -hide_banner -ss 00:56.0 -i "$inputFile" -vframes 1 -q:v 1 -y "/content/frames/frame1.png"
!curl --silent -F "reqtype=fileupload" -F "fileToUpload=@frame1.png" https://catbox.moe/user/api.php -o frame1.txt
f1 = open('frame1.txt', 'r')
%cd "/content"
file_content1 = f1.read()

%cd "/content/frames"
!ffmpeg -hide_banner -ss 02:20.0 -i "$inputFile" -vframes 1 -q:v 1 -y "/content/frames/frame2.png"
!curl --silent -F "reqtype=fileupload" -F "fileToUpload=@frame2.png" https://catbox.moe/user/api.php -o frame2.txt
%cd "/content/frames"
f2 = open('frame2.txt', 'r')
%cd "/content"
file_content2 = f2.read()

clear_output()
print ("Screenshot URLs:")
print ("1. " + file_content1)
print ("2. " + file_content2)


###__MediaInfo__

In [ ]:
path = "" #@param {type:"string"}
save_txt = True #@param {type:"boolean"}
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output, drive

def mediainfo():
  display(HTML("<br>"))
#   print(path.split("/")[::-1][0])
  display(HTML("<br>"))
#   media = !mediainfo "$path"
#   media = "\n".join(media).replace(os.path.dirname(path)+"/", "")
  get_ipython().system_raw("""mediainfo --LogFile="/root/.nfo" "$path" """)
  with open('/root/.nfo', 'r') as file:
    media = file.read()
    media = media.replace(os.path.dirname(path)+"/", "")
  print(media)
  get_ipython().system_raw("rm -f '/root/.nfo'")
  
  if save_txt:
    txt = path.rpartition('.')[0] + ".txt"
    if os.path.exists(txt):
      get_ipython().system_raw("rm -f '$txt'")
    !curl -s https://pastebin.com/raw/TV8Byydt -o "$txt"
    with open(txt, 'a+') as file:
      file.write("\n\n")
      file.write(media)

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output()
    
if not os.path.exists("/usr/bin/mediainfo"):
  get_ipython().system_raw("apt-get install mediainfo")
  
mediainfo()